In [2]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:




df = pd.read_csv('../../../data/processed/light.csv')
# Filter
timestamps = df.year.to_list()
texts = df.text.to_list()



In [4]:
def bert_text_preparation(texts, tokenizer, max_seq_length=612):
    """Preparing the input for BERT
    
    Takes a list of strings (texts) and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. Each sentence is treated as a separate segment.
    
    Args:
        texts (list): List of strings (sentences/documents) to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        max_seq_length (int): Maximum sequence length supported by the BERT model
        
    Returns:
        list: List of lists of BERT-readable tokens for each sentence
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
    tokenized_texts = []
    tokens_tensors = []
    segments_tensors = []

    for text in texts:
        # Calculate how much to truncate from the beginning and end
        truncate_length = len(text) - max_seq_length + 2  # +2 to account for [CLS] and [SEP]

        # Truncate the beginning and end of the text
        truncated_text = text[truncate_length//2 : -truncate_length//2]

        marked_text = "[CLS] " + truncated_text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(indexed_tokens)

        tokenized_texts.append(tokenized_text)
        tokens_tensors.append(indexed_tokens)
        segments_tensors.append(segments_ids)

    # Pad sequences to max_seq_length
    tokens_tensors = torch.nn.utils.rnn.pad_sequence([torch.tensor(t) for t in tokens_tensors], batch_first=True)
    segments_tensors = torch.nn.utils.rnn.pad_sequence([torch.tensor(s) for s in segments_tensors], batch_first=True)

    return tokenized_texts, tokens_tensors, segments_tensors


In [5]:
model = BertModel.from_pretrained('bert-base-uncased', 
                                  output_hidden_states = True)


In [6]:
# Prepare input using the bert_text_preparation function
tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(texts, tokenizer)



In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

# Selecting the output embeddings from the last layer
token_embeddings = hidden_states[-1]

# Assuming you want the embeddings for the first word in the first sentence
word_index = tokenized_texts[0].index("sovereignty")
word_embedding = token_embeddings[0, word_index].numpy()


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming you have the BERT model loaded and the embeddings for each word
# Example: word_embedding_sovereignty and word_embedding_territorial

# Reshape embeddings to be 2D arrays
embedding_sovereignty = word_embedding_sovereignty.reshape(1, -1)
embedding_territorial = word_embedding_territorial.reshape(1, -1)

# Calculate cosine similarity
cosine_sim = cosine_similarity(embedding_sovereignty, embedding_territorial)

print("Cosine Similarity between 'sovereignty' and 'territorial integrity':", cosine_sim[0][0])


In [ ]:
import plotly.express as px

# Assuming you have the BERT model loaded and the embeddings for each word at different time points
# Example: word_embedding_sovereignty_list and word_embedding_territorial_list

# Placeholder lists to store distances and time points
distances = []
time_points = []

# Collect embeddings at different time points
for time_point in range(num_time_points):
    # Example: Obtain word embeddings at different time points
    word_embedding_sovereignty = get_word_embedding_at_time_point("sovereignty", time_point)
    word_embedding_territorial = get_word_embedding_at_time_point("territorial integrity", time_point)

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(word_embedding_sovereignty.reshape(1, -1), word_embedding_territorial.reshape(1, -1))

    # Append distance and time point
    distances.append(cosine_sim[0][0])
    time_points.append(time_point)

# Create a Plotly figure
fig = px.line(x=time_points, y=distances, markers=True, labels={'x': 'Time Points', 'y': 'Cosine Similarity'},
              title='Cosine Similarity between "sovereignty" and "territorial integrity" over Time')

# Show the interactive plot
fig.show()


# Fast Tokenizer

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

model = BertModel.from_pretrained('bert-base-uncased', 
                                  output_hidden_states = True)


In [5]:
def bert_text_preparation(texts, tokenizer, max_seq_length=512):
    tokenized_texts = []
    tokens_tensors = []
    attention_masks = []

    for text in texts:
        # Tokenize the text
        tokenized_text = tokenizer.tokenize(text)

        truncate_length = len(text) - max_seq_length + 2  # +2 to account for [CLS] and [SEP]

        # Truncate the beginning and end of the text
        truncated_text = text[truncate_length//2 : -truncate_length//2]

        marked_text = "[CLS] " + truncated_text + " [SEP]"
        # Add special tokens [CLS] and [SEP]
        marked_text = ["[CLS]"] + tokenized_text + ["[SEP]"]

        # Convert tokens to ids
        indexed_tokens = tokenizer.convert_tokens_to_ids(marked_text)

        # Create attention mask
        attention_mask = [1] * len(indexed_tokens)

        # Pad sequences to max_seq_length
        while len(indexed_tokens) < max_seq_length:
            indexed_tokens.append(0)
            attention_mask.append(0)

        # Convert lists to PyTorch tensors
        tokens_tensors.append(torch.tensor(indexed_tokens))
        attention_masks.append(torch.tensor(attention_mask))

        tokenized_texts.append(tokenized_text)

    # Convert lists to PyTorch tensors
    tokens_tensors = torch.stack(tokens_tensors)
    attention_masks = torch.stack(attention_masks)

    return tokenized_texts, tokens_tensors, attention_masks


In [ ]:
# Prepare input using the bert_text_preparation function
tokenized_text, tokens_tensor, attention_masks = bert_text_preparation(texts, tokenizer)



Token indices sequence length is longer than the specified maximum sequence length for this model (628 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# Run the BERT model
with torch.no_grad():
    outputs = model(input_ids=tokens_tensor.view(-1, tokens_tensor.size(-1)), attention_mask=attention_masks.view(-1, attention_masks.size(-1)))
    

# Selecting the output embeddings from the last layer
#token_embeddings = hidden_states[-1]

# Assuming you want the embeddings for the first word in the first sentence
#word_index = tokenized_texts[0].index("sovereignty")
#word_embedding = token_embeddings[0, word_index].numpy()
